In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt # plotting
import PdM_functions as pdm # custom functions created for this project that are used in several notebooks

In [95]:
# Load the data
telemetry = pd.read_csv('data/PdM Azure/PdM_telemetry.csv', sep = ",")
errors = pd.read_csv('data/PdM Azure/PdM_errors.csv', sep = ",")
failures = pd.read_csv('data/PdM Azure/PdM_failures.csv', sep = ",")
machines = pd.read_csv('data/PdM Azure/PdM_machines.csv', sep = ",")
maint = pd.read_csv('data/PdM Azure/PdM_maint.csv', sep = ",")

print(telemetry)
print(errors)
print(failures)
print(machines)
print(maint)

                   datetime  machineID        volt      rotate    pressure  \
0       2015-01-01 06:00:00          1  176.217853  418.504078  113.077935   
1       2015-01-01 07:00:00          1  162.879223  402.747490   95.460525   
2       2015-01-01 08:00:00          1  170.989902  527.349825   75.237905   
3       2015-01-01 09:00:00          1  162.462833  346.149335  109.248561   
4       2015-01-01 10:00:00          1  157.610021  435.376873  111.886648   
...                     ...        ...         ...         ...         ...   
876095  2016-01-01 02:00:00        100  179.438162  395.222827  102.290715   
876096  2016-01-01 03:00:00        100  189.617555  446.207972   98.180607   
876097  2016-01-01 04:00:00        100  192.483414  447.816524   94.132837   
876098  2016-01-01 05:00:00        100  165.475310  413.771670  104.081073   
876099  2016-01-01 06:00:00        100  171.336037  496.096870   79.095538   

        vibration  
0       45.087686  
1       43.413973  
2  

In [98]:
machine1 = telemetry.loc[telemetry['machineID'] == 1,:]
failure1 = failures.loc[failures['machineID'] == 1,:].drop('machineID', axis = 1)
error1 = errors.loc[errors['machineID'] == 1,:].drop('machineID', axis = 1)
maint1 = maint.loc[maint['machineID'] == 1,:].drop('machineID', axis = 1)
detail1 =  machines.loc[machines['machineID'] == 1,:]
machine1 = pd.merge(machine1, failure1, on = 'datetime', how = 'left')
machine1['failure'].replace({np.nan: 'Normal'}, inplace=True)
machine1 = pd.merge(machine1, error1, on = 'datetime', how = 'left')
machine1['errorID'].replace({np.nan: 'Normal'}, inplace=True)
machine1 = pd.merge(machine1, maint1, on = 'datetime', how = 'left')
machine1['comp'].replace({np.nan: 'No'}, inplace=True)
machine1 = pd.merge(machine1, detail1, on = 'machineID', how = 'left')
machine1.rename(columns = {'comp': 'maintenance'})

,datetime,machineID,volt,rotate,pressure,vibration,failure,errorID,maintenance,model,age
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686,Normal,Normal,No,model3,18
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973,Normal,Normal,No,model3,18
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847,Normal,Normal,No,model3,18
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144,Normal,Normal,No,model3,18
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511,Normal,Normal,No,model3,18
...,...,...,...,...,...,...,...,...,...,...,...
8767,2016-01-01 02:00:00,1,153.724780,450.254337,85.197443,43.698137,Normal,Normal,No,model3,18
8768,2016-01-01 03:00:00,1,172.418099,375.701907,95.568265,42.111661,Normal,Normal,No,model3,18
8769,2016-01-01 04:00:00,1,174.588145,460.505353,95.483913,33.293572,Normal,Normal,No,model3,18
8770,2016-01-01 05:00:00,1,184.224228,438.823315,102.694358,46.609515,Normal,Normal,No,model3,18


In [99]:
#machine1.loc[machine1['failure'] != 'Normal',:]

In [100]:
#machine1.loc[machine1['errorID'] != 'Normal',:]